In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ------------------------------------ 451.7/451.7 kB 614.0 kB/s eta 0:00:00
     ------------------------------------ 139.5/139.5 kB 516.9 kB/s eta 0:00:00
     ------------------------------------ 134.3/134.3 kB 611.2 kB/s eta 0:00:00


In [4]:
from datasets import load_dataset

#dataset = load_dataset("yelp_review_full")
# df_train = pd.read_csv('/content/train.csv')
# df_test = pd.read_csv('/content/test.csv')


# dataset = load_dataset('csv', data_files='/content/train.csv')
# #dataset["train"][100]

In [6]:
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("Dataset/SentimentAnalysisofTweetsthroughAltmetrics/withheader", data_files=data_files)

Using custom data configuration withheader-2b3db70cca54906f


Dataset csv downloaded and prepared to C:/Users/asif.raza/.cache/huggingface/datasets/csv/withheader-2b3db70cca54906f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 328.46it/s]


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 733
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 315
    })
})

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

100%|██████████| 1/1 [00:00<00:00, 12.55ba/s]


In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)#.select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)#.select(range(1000))

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

Using TensorFlow backend.


In [12]:
!pip install evaluate

     -------------------------------------- 72.9/72.9 kB 444.5 kB/s eta 0:00:00


In [13]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
                                  
                                  
                                  )

In [16]:
# Log in to your Hugging Face account 
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\asif.raza\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 733
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 184
                                                       
Generating test split: 0 examples [25:43, ? examples/s]

{'loss': 1.1092, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.11}


                                                       
Generating test split: 0 examples [30:32, ? examples/s]

{'loss': 1.1071, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.22}


                                                       
Generating test split: 0 examples [35:51, ? examples/s]

...

                                                         
Generating test split: 0 examples [1:49:28, ? examples/s]

{'loss': 0.6903, 'learning_rate': 1.8e-05, 'epoch': 1.96}




Training completed. Do not forget to share your model on huggingface.co/models =)


                                                         
100%|██████████| 184/184 [1:30:17<00:00, 29.44s/it]les/s]

{'train_runtime': 5417.839, 'train_samples_per_second': 0.271, 'train_steps_per_second': 0.034, 'train_loss': 0.9465113463609115, 'epoch': 2.0}


TrainOutput(global_step=184, training_loss=0.9465113463609115, metrics={'train_runtime': 5417.839, 'train_samples_per_second': 0.271, 'train_steps_per_second': 0.034, 'train_loss': 0.9465113463609115, 'epoch': 2.0})